In [ ]:
# Utility to generate a MC4 sample for LOLA pretraining
# Please note: We have modified the python code in bertin-project/mc4-sampling locally for it to work
# More info: https://huggingface.co/datasets/bertin-project/mc4-sampling/discussions/6

In [1]:
from datasets import load_dataset
from datasets import concatenate_datasets
from datasets import Dataset
import pandas as pd
from tqdm import tqdm
# Uncomment to download the library files from huggingface
# load_dataset(
#         "bertin-project/mc4-sampling", languages=['en'],
#         split="train",
#         streaming=True,
#         sampling_method="random",
#         factor=0.5,
#     )

/local/upb/users/n/nikit/profiles/unix/cs/repos/LOLA-Megatron-DeepSpeed/venv-lola/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
langs = ["en","de","es","fr","hi","zh", "ja", "pt","ar", "ru"]
limit = 450000 # To create a dataset of 4.5M entries in total

In [3]:
def mc4_iterable(lang):

    return load_dataset(
        "bertin-project/mc4-sampling", languages=[lang],
        split="train",
        streaming=True,
        sampling_method="random",
        factor=0.5,
    )

In [4]:
lang_ds_arr = []

for lang in tqdm(langs):
    lang_ds = mc4_iterable(lang)
    data_list = list(lang_ds.take(limit))
    lang_ds = Dataset.from_pandas(pd.DataFrame(data=data_list))
    lang_ds_arr.append(lang_ds)

final_ds = concatenate_datasets(lang_ds_arr).shuffle()
print(final_ds)

  0%|          | 0/10 [00:00<?, ?it/s]/local/upb/users/n/nikit/profiles/unix/cs/repos/LOLA-Megatron-DeepSpeed/venv-lola/lib/python3.11/site-packages/datasets/builder.py:342: FutureWarning: Parameter 'name' was renamed to 'config_name' in version 2.3.0 and will be removed in 3.0.0.
  warnings.warn(
 10%|█         | 1/10 [02:06<19:01, 126.80s/it]/local/upb/users/n/nikit/profiles/unix/cs/repos/LOLA-Megatron-DeepSpeed/venv-lola/lib/python3.11/site-packages/datasets/builder.py:342: FutureWarning: Parameter 'name' was renamed to 'config_name' in version 2.3.0 and will be removed in 3.0.0.
  warnings.warn(
100%|██████████| 10/10 [1:25:02<00:00, 510.21s/it]


Dataset({
    features: ['text', 'timestamp', 'url'],
    num_rows: 4500000
})


In [5]:
final_ds

Dataset({
    features: ['text', 'timestamp', 'url'],
    num_rows: 4500000
})

In [6]:
final_ds.to_json(f"mc4-sample-4pt5m.jsonl", orient="records", lines=True, force_ascii=False)

Creating json from Arrow format: 100%|██████████| 4500/4500 [04:29<00:00, 16.69ba/s]


21731577196

**Look into lola_ws/gpt/preprocess_data.sh for the next steps**